In [10]:
%load_ext autotime
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm
import os

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 698 µs (started: 2023-01-21 01:08:22 -05:00)


## Commodity Translation File

In [11]:
commodityTranslator = pd.read_excel('API Resources/commodity_translation_wizard.xlsb', engine='pyxlsb',sheet_name='Import Concordance')
commodityTranslator = commodityTranslator.rename({'hts10':'HS10'},axis=1)
commodityTranslator['HS10'] = commodityTranslator['HS10'].apply(lambda x:str(x).zfill(10))
commodityTranslator.isna().sum()

year                      0
census_issue_date         0
HS10                      0
description_long          0
description_short         0
quantity_1                0
quantity_2           415902
sitc                      0
end_use                   0
naics                126040
ag_code                   0
hitech               177671
sic                  327442
dtype: int64

time: 51.3 s (started: 2023-01-21 01:08:23 -05:00)


In [12]:
# intCols = ['GEN_VAL_MO','CNT_VAL_MO','AIR_VAL_MO','CAL_DUT_MO','CNT_WGT_MO','AIR_WGT_MO']
def cleaningResponses(df):
    df = df[df['CTY_NAME']=='INDIA']
    df = df.drop('Unnamed: 0',axis=1)
    df['MONTH'] = df['MONTH'].astype(str).apply(lambda x:str(x).zfill(2))
    df['YearMonth'] = df[['YEAR','MONTH']].astype(str).agg('-'.join,axis=1)
    # df[intCols] = df[intCols].astype(int)
    if 'NACIS' in df.columns:
        df['NAICS'] = df['NAICS'].apply(lambda x:str(x).zfill(6))
        df['NA2'] = df['NACIS'].apply(lambda x:str(x)[:2])
        df['NA4'] = df['NACIS'].apply(lambda x:str(x)[:4])
        df = df.rename({'NACIS':'NA6'},axis=1)
    

    if 'I_COMMODITY' in df.columns:
        df['I_COMMODITY'] = df['I_COMMODITY'].apply(lambda x:str(x).zfill(10))
        df['HS2'] = df['I_COMMODITY'].apply(lambda x:str(x)[:2])
        df['HS4'] = df['I_COMMODITY'].apply(lambda x:str(x)[:4])
        df['HS6'] = df['I_COMMODITY'].apply(lambda x:str(x)[:6])
        df = df.rename({'I_COMMODITY':'HS10'},axis=1)
        # df = df[['YearMonth','HS2','HS4','HS6','HS10','I_COMMODITY_SDESC','DIST_NAME','CTY_NAME','GEN_VAL_MO','CNT_VAL_MO','AIR_VAL_MO','CAL_DUT_MO','CNT_WGT_MO','AIR_WGT_MO']]
    # dfFiltered
    return df

time: 1.25 ms (started: 2023-01-21 01:09:14 -05:00)


### Loading Saved Files 

In [13]:
hslistDf = []
naicslistDf = []
for fileName in tqdm(os.listdir('.apiResponses/hs')):
    if 'HSCYDT_2022' in fileName:hslistDf.append(pd.read_csv(os.path.join('.apiResponses/hs',fileName),compression='gzip'))
for fileName in tqdm(os.listdir('.apiResponses/naics')):
    if 'NACYDT_2022' in fileName:naicslistDf.append(pd.read_csv(os.path.join('.apiResponses/naics',fileName),compression='gzip'))

hscydtDf = cleaningResponses(pd.concat(hslistDf))
naicscydtDf = cleaningResponses(pd.concat(naicslistDf))

  1%|          | 1/116 [00:00<00:15,  7.57it/s]/tmp/ipykernel_51606/2232776438.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if 'NACYDT_2022' in fileName:naicslistDf.append(pd.read_csv(os.path.join('.apiResponses/naics',fileName),compression='gzip'))
100%|██████████| 116/116 [00:01<00:00, 67.27it/s]


time: 27.3 s (started: 2023-01-21 01:09:14 -05:00)


In [33]:
naicscydtDf.nunique()

NAICS             359
NAICS_LDESC       359
NAICS_SDESC       359
CTY_CODE            1
DISTRICT           43
DIST_NAME          43
GEN_CHA_MO      29061
GEN_VAL_MO      46324
GEN_CIF_MO      47373
CC_MO            1147
CNT_CHA_MO      25186
CNT_VAL_MO      36142
CNT_WGT_MO      30377
AIR_CHA_MO      11059
AIR_VAL_MO      19216
AIR_WGT_MO       7797
VES_CHA_MO      25717
VES_VAL_MO      37136
VES_WGT_MO      30940
CTY_NAME            1
YEAR                1
MONTH              11
COMM_LVL            1
SUMMARY_LVL2        1
YearMonth          11
dtype: int64

time: 109 ms (started: 2023-01-21 13:57:15 -05:00)


In [34]:
commodityTranslator.nunique()

year                    31
census_issue_date       27
HS10                 31580
description_long     29321
description_short    31448
quantity_1              51
quantity_2              21
sitc                  3239
end_use                142
naics                  549
ag_code                  2
hitech                  11
sic                    463
dtype: int64

time: 512 ms (started: 2023-01-21 13:57:32 -05:00)


In [30]:
hsNaicsMap = commodityTranslator[['HS10','description_long','description_short','quantity_1','naics']].drop_duplicates()
hsNaicsMap = hsNaicsMap.dropna()
hsNaicsMap.groupby('HS10')['naics'].nunique().reset_index().sort_values('naics',ascending=False)['naics'].value_counts()

1    18667
2     7493
3       62
Name: naics, dtype: int64

time: 693 ms (started: 2023-01-21 13:56:39 -05:00)


In [25]:
hscydtDf.merge(hsNaicsMap,how='left')

,HS10,I_COMMODITY_SDESC,I_COMMODITY_LDESC,CTY_CODE,DISTRICT,DIST_NAME,RP,GEN_CHA_MO,GEN_VAL_MO,GEN_CIF_MO,...,COMM_LVL,SUMMARY_LVL2,YearMonth,HS2,HS4,HS6,description_long,description_short,quantity_1,naics
0,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,10,"NEW YORK CITY, NY",-,13860,127179,141039,...,HS10,HSCYDT,2022-01,84,8421,842119,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",NO,333999
1,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,10,"NEW YORK CITY, NY",-,13860,127179,141039,...,HS10,HSCYDT,2022-01,84,8421,842119,"CENTRIFUGES, NESOI","CENTRIFUGES, NESOI",NO,333999
2,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,52,"MIAMI, FL",-,244,12479,12723,...,HS10,HSCYDT,2022-01,84,8421,842119,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",NO,333999
3,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,52,"MIAMI, FL",-,244,12479,12723,...,HS10,HSCYDT,2022-01,84,8421,842119,"CENTRIFUGES, NESOI","CENTRIFUGES, NESOI",NO,333999
4,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,53,"HOUSTON-GALVESTON, TX",-,241,135155,135396,...,HS10,HSCYDT,2022-01,84,8421,842119,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",NO,333999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919093,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,28,"SAN FRANCISCO, CA",-,0,0,0,...,HS10,HSCYDT,2022-11,11,1105,110510,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",KG,311211
919094,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,28,"SAN FRANCISCO, CA",-,0,0,0,...,HS10,HSCYDT,2022-11,11,1105,110510,FLOUR AND MEAL OF POTATOES,FLOUR AND MEAL OF POTATOES,KG,311211
919095,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,39,"CHICAGO, IL",-,1000,5673,6673,...,HS10,HSCYDT,2022-11,11,1105,110510,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",KG,311211
919096,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,39,"CHICAGO, IL",-,1000,5673,6673,...,HS10,HSCYDT,2022-11,11,1105,110510,FLOUR AND MEAL OF POTATOES,FLOUR AND MEAL OF POTATOES,KG,311211


time: 1.93 s (started: 2023-01-21 13:50:00 -05:00)


In [14]:
naicscydtDf

,NAICS,NAICS_LDESC,NAICS_SDESC,CTY_CODE,DISTRICT,DIST_NAME,GEN_CHA_MO,GEN_VAL_MO,GEN_CIF_MO,CC_MO,...,AIR_WGT_MO,VES_CHA_MO,VES_VAL_MO,VES_WGT_MO,CTY_NAME,YEAR,MONTH,COMM_LVL,SUMMARY_LVL2,YearMonth
23116,111110,SOYBEANS,SOYBEANS,5330,13,"BALTIMORE, MD",112700,769300,882000,7,...,0,112700,769300,660200,INDIA,2022,01,NA6,NACYDT,2022-01
23117,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,9,"BUFFALO, NY",74,22078,22152,1,...,0,0,0,0,INDIA,2022,01,NA6,NACYDT,2022-01
23118,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,4,"BOSTON, MA",710,4748,5458,1,...,0,710,4748,1212,INDIA,2022,01,NA6,NACYDT,2022-01
23119,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,7,"OGDENSBURG, NY",390,29512,29902,3,...,0,0,0,0,INDIA,2022,01,NA6,NACYDT,2022-01
23120,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,10,"NEW YORK CITY, NY",154429,1409019,1563448,59,...,100,154339,1406908,719305,INDIA,2022,01,NA6,NACYDT,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47271,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,41,"CLEVELAND, OH",2575,15565,18140,3,...,308,0,0,0,INDIA,2022,11,NA6,NACYDT,2022-11
47272,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,45,"ST. LOUIS, MO",0,0,0,0,...,0,0,0,0,INDIA,2022,11,NA6,NACYDT,2022-11
47273,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,53,"HOUSTON-GALVESTON, TX",1,17571,17572,1,...,0,1,17571,1833,INDIA,2022,11,NA6,NACYDT,2022-11
47274,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,55,"DALLAS-FORT WORTH, TX",15000,109117,124117,2,...,7698,0,0,0,INDIA,2022,11,NA6,NACYDT,2022-11


time: 91.1 ms (started: 2023-01-21 01:10:20 -05:00)


### HS Description Files

In [4]:
hs2Descriptions = pd.read_csv("API Resources/HS2 Descriptions.csv")
hs2Descriptions['HS2'] = hs2Descriptions['HS2'].apply(lambda x:str(x).zfill(2))
hs4Descriptions = pd.read_csv("API Resources/HS4 Descriptions.csv")
hs4Descriptions['HS4'] = hs4Descriptions['HS4'].apply(lambda x:str(x).zfill(4))
hs4Descriptions['HS2'] = hs4Descriptions['HS4'].apply(lambda x:str(x)[:2])
hs4Descriptions = hs4Descriptions.merge(hs2Descriptions,how='left')#.groupby(['HS2','HS2 Description'])['HS4'].count().reset_index().sort_values('HS4',ascending=True)\
    # .plot.barh(x='HS2 Description',y='HS4',figsize=(20, 30),title="HS4 Codes per HS2")

time: 194 ms (started: 2023-01-21 01:02:37 -05:00)


In [5]:
hscydtDf.columns

Index(['HS10', 'I_COMMODITY_SDESC', 'I_COMMODITY_LDESC', 'CTY_CODE',
       'DISTRICT', 'DIST_NAME', 'RP', 'GEN_CHA_MO', 'GEN_VAL_MO', 'GEN_CIF_MO',
       'GEN_QY1_MO', 'UNIT_QY1', 'CC_MO', 'CNT_CHA_MO', 'CNT_VAL_MO',
       'CNT_WGT_MO', 'AIR_CHA_MO', 'AIR_VAL_MO', 'AIR_WGT_MO', 'VES_CHA_MO',
       'VES_VAL_MO', 'VES_WGT_MO', 'CTY_NAME', 'YEAR', 'MONTH', 'COMM_LVL',
       'SUMMARY_LVL2', 'YearMonth', 'HS2', 'HS4', 'HS6'],
      dtype='object')

time: 9.99 ms (started: 2023-01-21 01:02:39 -05:00)


In [6]:
hscydtDf

,HS10,I_COMMODITY_SDESC,I_COMMODITY_LDESC,CTY_CODE,DISTRICT,DIST_NAME,RP,GEN_CHA_MO,GEN_VAL_MO,GEN_CIF_MO,...,VES_WGT_MO,CTY_NAME,YEAR,MONTH,COMM_LVL,SUMMARY_LVL2,YearMonth,HS2,HS4,HS6
4,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,10,"NEW YORK CITY, NY",-,13860,127179,141039,...,2721,INDIA,2022,01,HS10,HSCYDT,2022-01,84,8421,842119
5,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,52,"MIAMI, FL",-,244,12479,12723,...,0,INDIA,2022,01,HS10,HSCYDT,2022-01,84,8421,842119
6,8421190000,"CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI","CENTRIFUGES, INCLUDING CENTRIFUGAL DRYERS, NESOI",5330,53,"HOUSTON-GALVESTON, TX",-,241,135155,135396,...,3056,INDIA,2022,01,HS10,HSCYDT,2022-01,84,8421,842119
54,8421390115,"DUST COLLECTION AND AIR PURIFICATION EQUIP, NESOI",DUST COLLECTION AND AIR PURIFICATION EQUIPMENT...,5330,9,"BUFFALO, NY",-,747,84443,85190,...,0,INDIA,2022,01,HS10,HSCYDT,2022-01,84,8421,842139
55,8421390115,"DUST COLLECTION AND AIR PURIFICATION EQUIP, NESOI",DUST COLLECTION AND AIR PURIFICATION EQUIPMENT...,5330,10,"NEW YORK CITY, NY",-,500,44928,45428,...,1100,INDIA,2022,01,HS10,HSCYDT,2022-01,84,8421,842139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425184,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,17,"SAVANNAH, GA",-,22600,86810,109410,...,121100,INDIA,2022,11,HS10,HSCYDT,2022-11,11,1105,110510
425185,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,27,"LOS ANGELES, CA",-,0,0,0,...,0,INDIA,2022,11,HS10,HSCYDT,2022-11,11,1105,110510
425186,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,28,"SAN FRANCISCO, CA",-,0,0,0,...,0,INDIA,2022,11,HS10,HSCYDT,2022-11,11,1105,110510
425187,1105100000,"FLOUR, MEAL AND POWDER OF POTATOES","FLOUR, MEAL AND POWDER OF POTATOES",5330,39,"CHICAGO, IL",-,1000,5673,6673,...,3125,INDIA,2022,11,HS10,HSCYDT,2022-11,11,1105,110510


time: 778 ms (started: 2023-01-21 01:02:40 -05:00)


## Total Imports Values HS2 by Year

In [41]:
hscydtINDDf = hscydtDf[hscydtDf['CTY_NAME']=='INDIA']
intCols = ['GEN_CHA_MO', 'GEN_VAL_MO', 'GEN_CIF_MO',
       'GEN_QY1_MO', 'CNT_CHA_MO', 'CNT_VAL_MO',
       'CNT_WGT_MO', 'AIR_CHA_MO', 'AIR_VAL_MO', 'AIR_WGT_MO', 'VES_CHA_MO',
       'VES_VAL_MO', 'VES_WGT_MO']
masterhsdf = hscydtINDDf.merge(hs4Descriptions,how='left')\
    .merge(commodityTranslator[['HS10',"description_long","description_short",'naics']].drop_duplicates(),how='left')
masterhsdf[intCols] = masterhsdf[intCols].astype(int)

# intCols = ['GEN_VAL_MO','CNT_VAL_MO','AIR_VAL_MO','CAL_DUT_MO','CNT_WGT_MO','AIR_WGT_MO']
# masterhsdf[intCols] = masterhsdf[intCols].astype(int)
hs2Summary = masterhsdf.groupby(['YEAR','HS2','HS2 Description'])[intCols].sum().reset_index()
hs2Summary.sort_values('YEAR',ascending=True)
hs2Summary.columns

# hs2Summary['Year'] = hs2Summary['YearMonth'].apply(lambda x:x.split('-')[0])
hs2SummaryYR = hs2Summary.groupby(['YEAR','HS2','HS2 Description'])[intCols].sum().reset_index()
hs2SummaryYR

,YEAR,HS2,HS2 Description,GEN_CHA_MO,GEN_VAL_MO,GEN_CIF_MO,GEN_QY1_MO,CNT_CHA_MO,CNT_VAL_MO,CNT_WGT_MO,AIR_CHA_MO,AIR_VAL_MO,AIR_WGT_MO,VES_CHA_MO,VES_VAL_MO,VES_WGT_MO
0,2022,01,LIVE ANIMALS,10827,20630,31457,35,529,7130,300,10298,13500,66,529,7130,300
1,2022,03,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",180443782,2133407461,2313851243,244503571,179268336,2123459915,291191781,360670,636681,43004,179738255,2127367160,291810481
2,2022,04,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,26836466,220996932,247833398,84690027,26822617,220805967,96180757,12049,156583,4718,26823917,220818921,96182105
3,2022,05,"PRODUCTS OF ANIMAL ORIGIN, NESOI",3365910,19101435,22467345,3706231,3245722,17478578,3814302,72665,738807,19454,3292440,17810742,3875040
4,2022,06,"LIVE TREES AND OTHER PLANTS; BULBS, ROOTS AND ...",6655013,37573158,44228171,59790047,3094489,15284292,3398721,3332218,13968370,918485,3270263,16009931,3511768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2022,95,"TOYS, GAMES AND SPORTS EQUIPMENT; PARTS AND AC...",69674978,679102383,748777361,238587867,63150850,606688669,96855264,5603885,59001123,2792336,63748454,611998024,97366030
93,2022,96,MISCELLANEOUS MANUFACTURED ARTICLES,37090979,396956665,434047644,2115999858,33705650,349119645,44236149,2945190,23005594,775907,33856232,353565430,44708280
94,2022,97,"WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",716804,54906812,55623616,295981,207978,2428393,318700,496622,50957960,64155,218639,3022787,324626
95,2022,98,"SPECIAL CLASSIFICATION PROVISIONS, NESOI",20347669,1097283438,1117631107,168826062,10387284,180277782,22613629,8969472,836439459,3086131,10583652,187447236,23216666


time: 4.99 s (started: 2023-01-21 00:56:53 -05:00)


In [15]:
import plotly.express as px

fig = px.bar(hs2Summary, x="YearMonth", y="GEN_VAL_MO", color="HS2 Description", 
                        width=1500, height=800,
                        title="Total Import Values from India 2022 by HS2 Code")
fig.update_layout(showlegend=True)
fig.show()

time: 402 ms (started: 2023-01-19 22:00:22 -05:00)


In [60]:
naicscydtDf

,NAICS,NAICS_LDESC,NAICS_SDESC,CTY_CODE,DISTRICT,DIST_NAME,GEN_CHA_MO,GEN_VAL_MO,GEN_CIF_MO,CC_MO,...,AIR_WGT_MO,VES_CHA_MO,VES_VAL_MO,VES_WGT_MO,CTY_NAME,YEAR,MONTH,COMM_LVL,SUMMARY_LVL2,YearMonth
23116,111110,SOYBEANS,SOYBEANS,5330,13,"BALTIMORE, MD",112700,769300,882000,7,...,0,112700,769300,660200,INDIA,2022,01,NA6,NACYDT,2022-01
23117,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,9,"BUFFALO, NY",74,22078,22152,1,...,0,0,0,0,INDIA,2022,01,NA6,NACYDT,2022-01
23118,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,4,"BOSTON, MA",710,4748,5458,1,...,0,710,4748,1212,INDIA,2022,01,NA6,NACYDT,2022-01
23119,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,7,"OGDENSBURG, NY",390,29512,29902,3,...,0,0,0,0,INDIA,2022,01,NA6,NACYDT,2022-01
23120,111120,OILSEEDS (EXCEPT SOYBEAN),OILSEEDS (EXCEPT SOYBEAN),5330,10,"NEW YORK CITY, NY",154429,1409019,1563448,59,...,100,154339,1406908,719305,INDIA,2022,01,NA6,NACYDT,2022-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47271,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,41,"CLEVELAND, OH",2575,15565,18140,3,...,308,0,0,0,INDIA,2022,11,NA6,NACYDT,2022-11
47272,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,45,"ST. LOUIS, MO",0,0,0,0,...,0,0,0,0,INDIA,2022,11,NA6,NACYDT,2022-11
47273,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,53,"HOUSTON-GALVESTON, TX",1,17571,17572,1,...,0,1,17571,1833,INDIA,2022,11,NA6,NACYDT,2022-11
47274,990000,OTHER SPECIAL CLASSIFICATION PROVISIONS,OTHER SPECIAL CLASSIFICATION PROVISIONS,5330,55,"DALLAS-FORT WORTH, TX",15000,109117,124117,2,...,7698,0,0,0,INDIA,2022,11,NA6,NACYDT,2022-11


time: 78.8 ms (started: 2023-01-21 14:32:46 -05:00)


In [65]:
naicscydtDf.columns

Index(['NAICS', 'NAICS_LDESC', 'NAICS_SDESC', 'CTY_CODE', 'DISTRICT',
       'DIST_NAME', 'GEN_CHA_MO', 'GEN_VAL_MO', 'GEN_CIF_MO', 'CC_MO',
       'CNT_CHA_MO', 'CNT_VAL_MO', 'CNT_WGT_MO', 'AIR_CHA_MO', 'AIR_VAL_MO',
       'AIR_WGT_MO', 'VES_CHA_MO', 'VES_VAL_MO', 'VES_WGT_MO', 'CTY_NAME',
       'YEAR', 'MONTH', 'COMM_LVL', 'SUMMARY_LVL2', 'YearMonth'],
      dtype='object')

time: 4.99 ms (started: 2023-01-21 14:41:13 -05:00)


In [68]:
intCols = ['GEN_CHA_MO','GEN_VAL_MO','AIR_CHA_MO','AIR_VAL_MO','CNT_CHA_MO','CNT_VAL_MO']
naicscydtDf[intCols] = naicscydtDf[intCols].astype(int)
naicsSummary = naicscydtDf.groupby(['NAICS_SDESC'])[intCols].sum().reset_index()
naicsSummary['Total_VAL_YR'] = naicsSummary['AIR_VAL_MO']+naicsSummary['CNT_VAL_MO']
naicsSummary['Total_CHA_YR'] = naicsSummary['AIR_CHA_MO']+naicsSummary['CNT_CHA_MO']
naicsSummary
    # .apply(lambda x: x.sort_values(['GEN_VAL_MO'],ascending=False))
        # .groupby(['YearMonth','NAICS_SDESC'])['GEN_VAL_MO'].sum()

,NAICS_SDESC,GEN_CHA_MO,GEN_VAL_MO,AIR_CHA_MO,AIR_VAL_MO,CNT_CHA_MO,CNT_VAL_MO,Total_VAL_YR,Total_CHA_YR
0,ABRASIVE PRODUCTS,1384599,23778283,202763,3010877,972554,16140413,19151290,1175317
1,AC/WARM AIR HTG & COMMERCIAL REFRIG EQUIP,2968875,64760747,462011,19919015,2437422,42894068,62813083,2899433
2,ADHESIVES,120847,1756808,5327,64576,113519,1679782,1744358,118846
3,AIR & GAS COMPRESSORS,14908926,264168071,2079546,34378565,12606650,218523893,252902458,14686196
4,AIRCRAFT,201458,991598,201458,991598,0,0,991598,201458
...,...,...,...,...,...,...,...,...,...
354,"WOMEN'S, GIRLS', AND INFANTS' CUT AND SEW APPAREL",161911574,3239106694,73419812,716498469,85834864,2468040699,3184539168,159254676
355,WOOD CONTAINERS & PALLETS,823056,8551632,13144,76213,799718,8254671,8330884,812862
356,WOOD KITCHEN CABINETS & COUNTERTOPS,271391,683175,167,4355,252649,568794,573149,252816
357,WOOD OFFICE FURNITURE,530298,3912753,39587,389689,486998,3461776,3851465,526585


time: 63 ms (started: 2023-01-21 14:52:29 -05:00)


In [ ]:
hs2Summary['YearMonth'].unique()

array(['2014-1', '2014-10', '2014-11', '2014-12', '2014-2', '2014-3',
       '2014-4', '2014-5', '2014-6', '2014-7', '2014-8', '2014-9',
       '2015-1', '2015-10', '2015-11', '2015-12', '2015-2', '2015-3',
       '2015-4', '2015-5', '2015-6', '2015-7', '2015-8', '2015-9',
       '2016-1', '2016-10', '2016-11', '2016-12', '2016-2', '2016-3',
       '2016-4', '2016-5', '2016-6', '2016-7', '2016-8', '2016-9',
       '2017-1', '2017-10', '2017-11', '2017-12', '2017-2', '2017-3',
       '2017-4', '2017-5', '2017-6', '2017-7', '2017-8', '2017-9',
       '2018-1', '2018-10', '2018-11', '2018-12', '2018-2', '2018-3',
       '2018-4', '2018-5', '2018-6', '2018-7', '2018-8', '2018-9',
       '2019-1', '2019-10', '2019-11', '2019-12', '2019-2', '2019-3',
       '2019-4', '2019-5', '2019-6', '2019-7', '2019-8', '2019-9',
       '2020-1', '2020-10', '2020-11', '2020-12', '2020-2', '2020-3',
       '2020-4', '2020-5', '2020-6', '2020-7', '2020-8', '2020-9',
       '2021-1', '2021-10', '2021-11', '2

time: 4.57 ms (started: 2023-01-19 21:06:51 -05:00)


In [24]:

import plotly.graph_objects as go

audi = df[df['car'] == 'Audi Q5']
benz = df[df['car'] == 'Merucedes GLE']
bmw = df[df['car'] == 'BMW X5']
x = df['date'].unique()

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=audi['Fuel Efficiency'],
                    mode='lines+markers',
                    name='Audi Q5', marker=dict(color="DarkOrange"))),
fig.add_trace(go.Scatter(x=x, y=benz['Fuel Efficiency'],
                    mode='lines+markers',
                    name='Merucedes GLE', marker=dict(color="Crimson"))),
fig.add_trace(go.Scatter(x=x, y=bmw['Fuel Efficiency'],
                    mode='lines+markers', name='BMW X5', marker=dict(color="RebeccaPurple")))

fig.update_layout(title_text='Fuel Efficiency Visual',
                 xaxis_title='Date', yaxis_title='FuelEfficiency')


fig.update_layout(
    updatemenus=[
        dict(active=0,
            buttons=list([
            dict(label="None",
                 method="update",
                 args=[{"visible":[True,True,True]},
                       {"title":"ALL"}]),
            dict(label="Audi Q5",
                 method="update",
                 args=[{"visible":[True, False, False]},
                       {"title":"Audi Q5"}]),
            dict(label="Merucedes GLE",
                 method="update",
                 args=[{"visible":[False,True,False]},
                       {"title":"Merucedes GLE"}]),
            dict(label="BMW X5",
                 method="update",
                 args=[{"visible":[False,False,True]},
                       {"title":"BMW X5"}])
        ]),
        )
    ]
)

fig.show()

NameError: name 'df' is not defined

time: 37.6 ms (started: 2023-01-21 00:02:02 -05:00)
